In [129]:
import yfinance as yf
import pandas as pd
import numpy as np

google = yf.Ticker("GOOG")
historical_data = google.history(period="5d")

# Print the columns of the dataframe 
print(historical_data.columns)

print(historical_data['Open'])

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')
Date
2024-02-13 00:00:00-05:00    146.070007
2024-02-14 00:00:00-05:00    147.369995
2024-02-15 00:00:00-05:00    144.460007
2024-02-16 00:00:00-05:00    144.210007
2024-02-20 00:00:00-05:00    140.940002
Name: Open, dtype: float64


In [15]:
def alpha_26(ticker, time_frame):
  """
  Calculates the Alpha#26 factor for a given ticker.

  Args:
    ticker: The ticker symbol of the stock.

  Returns:
    The Alpha#26 factor value.
  """

#   Alpha26: (-1 * ts_max(correlation(ts_rank(volume, 5), ts_rank(high, 5), 5), 3))

  # Download the historical stock data, where time_frame is a window of time, like "5y" or "1y"
  data = yf.download(ticker, period=time_frame)
  # Calculate the volume and high price ranks.
  volume_rank = data['Volume'].rolling(window=5).apply(lambda x: pd.Series(x).rank(method='first').iloc[-1])
  high_rank = data['High'].rolling(window=5).apply(lambda x: pd.Series(x).rank(method='first').iloc[-1])

  # Calculate the correlation between volume and high price ranks.
  correlation = volume_rank.rolling(window=5).corr(high_rank)

  # Calculate the ts_max.
  ts_max = correlation.rolling(window=3).max()

  # Calculate the Alpha#26 factor.
  alpha_26 = -1 * ts_max

  return alpha_26.iloc[-1]

In [16]:
def get_vwap(data):
    # data = yf.download(ticker, period=time_frame, interval="1m")

    # Calculate VWAP
    data['TP'] = (data['High'] + data['Low'] + data['Close']) / 3
    data['CumVol'] = data['Volume'].cumsum()
    data['CumTP'] = (data['TP'] * data['Volume']).cumsum()
    data['VWAP'] = data['CumTP'] / data['CumVol']

    return(data['VWAP'])

In [23]:
def alpha_27(ticker, time_frame):
    # Calculate the rank of volume and VWAP
    data = yf.download(ticker, period=time_frame)
    volume_rank = data['Volume'].rank(method='first')
    vwap_rank = get_vwap(data).rank(method='first')

    # Calculate the correlation between rank(volume) and rank(VWAP) over a window of 6
    correlation_rank = volume_rank.rolling(window=6).corr(vwap_rank)

    # Calculate the sum of the correlation over a window of 2 and divide by 2.0
    sum_corr = correlation_rank.rolling(window=2).sum() / 2.0

    # Calculate the rank of sum_corr
    rank_sum_corr = sum_corr.rank(method='first')

    # Check if 0.5 < rank_sum_corr and assign -1 or 1 accordingly
    alpha_27 = rank_sum_corr.apply(lambda x: -1 if x > 0.5 else 1)

    return alpha_27.iloc[-1]  # Return the last value of the Alpha#27 factor

In [24]:
def min_max_scale(value, min_val, max_val):
    # Scale the value to be between 0 and 1
    return (value - min_val) / (max_val - min_val)

In [27]:
def alpha_28(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, time_frame)

    # Calculate ADV20
    data['ADV20'] = data['Volume'].rolling(window=20).mean()

    # Calculate correlation between ADV20 and Low over 5 periods
    corr_adv_low = data['ADV20'].rolling(window=5).corr(data['Low'])

    # Calculate average price
    avg_price = (data['High'] + data['Low']) / 2

    # Calculate Alpha#28 with scaling
    alpha_28 = ((corr_adv_low + avg_price) - data['Close']).mean()

    # I scale the value here, though I'm not sure which kind of scaling to use.
    min_val = alpha_28.min()
    max_val = alpha_28.max()
    scaled_alpha_28 = alpha_28.apply(lambda x: min_max_scale(x, min_val, max_val))

    return scaled_alpha_28

In [419]:
def alpha_29(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate necessary intermediate values
    delta_close = data['Close'].diff(1)
    rank_1 = delta_close.rank(axis=0, method='first')
    rank_2 = rank_1.rank(axis=0, method='first')
    rank_3 = rank_2 * -1
    rank_4 = rank_3.rolling(window=5).sum()
    rank_5 = rank_4.rank(axis=0, method='first')
    rank_6 = rank_5.rank(axis=0, method='first')
    ts_min_1 = rank_6.rolling(window=2).min()
    sum_1 = ts_min_1.sum(axis=0)
    log_1 = np.log(sum_1)
    scale_1 = rank_1 * log_1
    rank_7 = scale_1.rank(axis=0, method='first')
    rank_8 = rank_7.rank(axis=0, method='first')
    product_1 = rank_8 * rank_7
    min_1 = product_1.rolling(window=5).min()
    min_rank = min_1.rank(axis=0, method='first')
    ts_rank_1 = (-1 * data['Close'].pct_change()).shift(-6).rolling(window=5).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    ts_rank_2 = ts_rank_1.rolling(window=5).mean()

    # Calculate Alpha#29
    alpha_29 = (min_rank + ts_rank_2).mean()

    return alpha_29

In [ ]:
(((1.0 - rank(((sign((close - delay(close, 1))) + sign((delay(close, 1) - delay(close, 2)))) + sign((delay(close, 2) - delay(close, 3)))))) * sum(volume, 5)) / sum(volume, 20))

In [266]:
def alpha_30(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)
    # print(data)

    close = data['Close']
    volume = data['Volume']

    # print(volume)

    # Calculate signs
    sign_1 = np.sign(close - close.shift(-1)).dropna()
    sign_2 = np.sign(close.shift(-1) - close.shift(-2)).dropna()
    sign_3 = np.sign(close.shift(-2) - close.shift(-3)).dropna()

    # print(sign_1)
    # print("Sirgn 2")
    # print(sign_2)
    # print("Done")

    # Calculate rank of the signs - WHY DO WE DO THAT
    rank_1 = sign_1.rank()
    rank_2 = sign_2.rank()
    rank_3 = sign_3.rank()

    # print(rank_1)

    # Calculate the first part of the formula
    part_1 = 1.0 - ((rank_1 + rank_2 + rank_3) / 3.0)

    print(part_1)

    # Calculate the sum of volume over 5 and 20 periods
    sum_volume_5 = volume.rolling(window=5).sum()
    sum_volume_20 = volume.rolling(window=20).sum()

    # Calculate Alpha#30
    alpha_30 = ((part_1 * sum_volume_5) / sum_volume_20).dropna()

    return alpha_30

In [217]:
((rank(rank(rank(decay_linear((-1 * rank(rank(delta(close, 10)))), 10)))) + rank((-1 * delta(close, 3)))) + sign(scale(correlation(adv20, low, 12))))

NameError: name 'rank' is not defined

In [145]:
# Not sure obout this
def decay_linear(x, d):
    weights = np.arange(d, 0, -1).astype('float64')
    weights /= weights.sum()
    return np.convolve(x, weights, mode='valid')

In [208]:
# STUCK ON THIS ONE, MOVE ON FOR THE TIME BEING
# def alpha_31(ticker, time_frame):

#     data = yf.download(ticker, period=time_frame)
    
#     delta_close_10 = data['Close'].diff(10)

#     # Calculate Rank of Delta Close
#     rank_delta_close_10 = delta_close_10.rank()

#     # Calculate Double Rank of Delta Close
#     neg_double_rank_delta_close_10 = -1 * rank_delta_close_10.rank()

#     decay_linear_double_rank_delta_close_10 = decay_linear(neg_double_rank_delta_close_10, 10)

#     rank_decay_linear_double_rank_delta_close_10 = decay_linear_double_rank_delta_close_10.rank()

#     part_1 = rank_decay_linear_double_rank_delta_close_10.rank()

#     neg_delta_close_3 = (-1 * data['Close'].diff(3))

#     rank_neg_delta_close_3 = neg_delta_close_3.rank()

#     part_2 = (part_1 + rank_neg_delta_close_3).rank() # first addition - this covers all the ranks at the beginning of the equation now

#     # Calculate Correlation between adv20 and low over 12 periods
#     correlation_adv20_low_12 = (data['Volume'].rolling(window=20).mean()).rolling(window=12).corr(data['Low'])

#     # Calculate Scale of Correlation
#     scale_correlation_adv20_low_12 = correlation_adv20_low_12.apply(lambda x: min_max_scale(x, correlation_adv20_low_12.min(), correlation_adv20_low_12.max()))

#     sign_scale_correlation_adv20_low_12 = np.sign(scale_correlation_adv20_low_12)

#     alpha_31 = part_2 + sign_scale_correlation_adv20_low_12

#     return alpha_31


# def scale_function(x):
#     return (x - x.min()) / (x.max() - x.min())


# def alpha_31(ticker, time_frame):
#     data = yf.download(ticker, period=time_frame)
    
#     # Calculate Rank of Delta Close
#     rank_delta_close_10 = data['Close'].diff(10).rank()
#     print("Rank of Delta:", rank_delta_close_10)

#     # Calculate Double Rank of Delta Close
#     neg_double_rank_delta_close_10 = -1 * rank_delta_close_10.rank()

#     # Calculate Linear Decay of Double Rank of Delta Close
#     decay_linear_double_rank_delta_close_10 = pd.Series(decay_linear(neg_double_rank_delta_close_10, 10))

#     # Calculate Triple Rank of Linear Decay of Double Rank of Delta Close
#     triple_rank_decay_linear_double_rank_delta_close_10 = decay_linear_double_rank_delta_close_10.rank()

#     # Calculate Rank of Triple Rank of Linear Decay of Double Rank of Delta Close
#     rank_triple_rank_decay_linear_double_rank_delta_close_10 = triple_rank_decay_linear_double_rank_delta_close_10.rank()

#     # Calculate Rank of Delta Close for the last 3 days
#     rank_neg_delta_close_3 = (-1 * data['Close'].diff(3)).rank()

#     print(rank_neg_delta_close_3)

#     # Calculate Rank of Correlation between adv20 and low over 12 periods
#     correlation_adv20_low_12 = data['Volume'].rolling(window=20).mean().rolling(window=12).corr(data['Low'])
#     rank_correlation_adv20_low_12 = correlation_adv20_low_12.rank()

#     # Calculate Scale of Correlation
#     scale_correlation_adv20_low_12 = scale(correlation_adv20_low_12).dropna()
#     print(scale_correlation_adv20_low_12)

#     # Calculate Sign of Scale of Correlation
#     sign_scale_correlation_adv20_low_12 = np.sign(scale_correlation_adv20_low_12).dropna()

#     print(rank_triple_rank_decay_linear_double_rank_delta_close_10.dtype)
#     print(rank_neg_delta_close_3.dtype)
#     print(sign_scale_correlation_adv20_low_12.dtype)

#     # Calculate Alpha#31
#     alpha_31 = (rank_triple_rank_decay_linear_double_rank_delta_close_10 +
#                 rank_neg_delta_close_3 +
#                 sign_scale_correlation_adv20_low_12)
    
#     alpha_31 = alpha_31.dropna()

#     return alpha_31

def alpha_31(ticker, time_frame):
    data = yf.download(ticker, period=time_frame)

    print(data.shape[0])
    # print(data['Close'])
    
    # Calculate rank of delta close
    delta_close_10 = data['Close'].diff(10).dropna() # these diff functions is where the NaN came from
    # print(delta_close_10)
    rank_delta_close_10 = delta_close_10.rank()

    # print("Checking: ", rank_delta_close_10)

    # Calculate double rank of delta close
    neg_double_rank_delta_close_10 = -1 * rank_delta_close_10.rank()

    # print("Checking: ", neg_double_rank_delta_close_10)

    # Calculate decay linear of double rank delta close
    decay_linear_double_rank_delta_close_10 = decay_linear(neg_double_rank_delta_close_10, 10)

    # print("Checking: ", decay_linear_double_rank_delta_close_10)

    # Calculate triple rank of decay linear of double rank delta close
    triple_rank_decay_linear_double_rank_delta_close_10 = pd.DataFrame(decay_linear_double_rank_delta_close_10).rank()

    # print("Checking: ", triple_rank_decay_linear_double_rank_delta_close_10)

    # Calculate rank of triple rank of decay linear of double rank delta close
    rank_triple_rank_decay_linear_double_rank_delta_close_10 = triple_rank_decay_linear_double_rank_delta_close_10.rank()

    # print("Checking: ", rank_triple_rank_decay_linear_double_rank_delta_close_10)

    # Calculate rank of delta close for the last 3 days
    rank_neg_delta_close_3 = (-1 * data['Close'].diff(3)).rank().dropna()

    # print("Checking delta rank: ", rank_neg_delta_close_3)

    # Calculate correlation between adv20 and low over 12 periods
    # print("Volume: ", data['Volume'])
    adv20_starter = (data['Volume'].dropna()).rolling(window=20)
    # print("Check me out: ", adv20_starter)
    adv20 = (data['Volume'].rolling(window=20).mean().dropna())
    # print("Checking adv20: ", adv20)
    correlation_adv20_low_12 = (adv20.rolling(window=12).corr(data['Low']))

    correlation_adv20_low_12 = correlation_adv20_low_12.dropna()

    # print("Checking: ", correlation_adv20_low_12)

    # print("Checking: ", correlation_adv20_low_12)

    # Calculate scale of correlation
    scale_correlation_adv20_low_12 = correlation_adv20_low_12.apply(lambda x: min_max_scale(x, correlation_adv20_low_12.min(), correlation_adv20_low_12.max()))

    # print("Checking now: ", scale_correlation_adv20_low_12)    

    # Calculate sign of scale of correlation
    sign_scale_correlation_adv20_low_12 = np.sign(scale_correlation_adv20_low_12)

    # print("Checking now: ", sign_scale_correlation_adv20_low_12)

    # if isinstance(rank_triple_rank_decay_linear_double_rank_delta_close_10, pd.Timestamp):
    #     print("Variable contains a timestamp")
    # else:
    #     print("Variable does not contain a timestamp")

    # if isinstance(rank_neg_delta_close_3, pd.Timestamp):
    #     print("Variable contains a timestamp")
    # else:
    #     print("Variable does not contain a timestamp")

    # if isinstance(sign_scale_correlation_adv20_low_12, pd.Timestamp):
    #     print("Variable contains a timestamp")
    # else:
    #     print("Variable does not contain a timestamp")

    alpha_31 = rank_triple_rank_decay_linear_double_rank_delta_close_10 + rank_neg_delta_close_3 + sign_scale_correlation_adv20_low_12

    print("Checking now: ", rank_triple_rank_decay_linear_double_rank_delta_close_10)
    print("Checking now: ", rank_neg_delta_close_3)

    print("Checking now: ", sign_scale_correlation_adv20_low_12)

    return alpha_31

def scale(x, a=1):
    total_abs = np.sum(np.abs(x))
    if total_abs == 0:
        return x
    return x * (a / total_abs)

In [413]:
def alpha_32(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    close = data['Close']
    vwap = get_vwap(data)

    # Calculate the first part of the formula
    part_1 = (close.rolling(window=7).sum() / 7 - close).mean()

    # Drop NaN values from vwap and delayed close
    vwap_dropna = vwap#.dropna()
    close_delayed_dropna = close.shift(-5)#.dropna()

    close_delayed_dropna = close_delayed_dropna#.dropna()

    # Calculate the second part of the formula
    # part_2 = 20 * np.corrcoef(vwap_dropna, close_delayed_dropna)[0, 1]

    # print("Correlation Check: ", vwap_dropna.corr(close_delayed_dropna))

    part_2 = 20 * vwap_dropna.corr(close_delayed_dropna)

    # print(part_2)

    # Scale the two parts and sum them
    alpha_32 = part_1 + part_2
    alpha_32 = min_max_scale(alpha_32.min(), alpha_32.max(), 230) # Does this make any sense?

    return alpha_32

In [327]:
def alpha_33(ticker, time_frame):
    data = yf.download(ticker, period=time_frame)
    open_price = data['Open']
    close_price = data['Close']
    alpha = -1 * ((1 - (open_price / close_price)) ** 1)
    return alpha.rank()

In [333]:
def alpha_34(ticker, time_frame):

    data = yf.download(ticker, period=time_frame)
    returns = data['Close'].pct_change()
    std_dev_2 = returns.rolling(window=2).std()
    std_dev_5 = returns.rolling(window=5).std()

    part_1 = 1 - std_dev_2.rank(pct=True) / std_dev_5.rank(pct=True)
    part_2 = 1 - returns.diff(1).rank(pct=True)

    alpha = (part_1 + part_2).rank()
    return alpha.dropna()

In [336]:
def alpha_35(ticker, time_frame):

    data = yf.download(ticker, period=time_frame)

    volume = data['Volume']
    close = data['Close']
    high = data['High']
    low = data['Low']
    returns = data['Close'].pct_change()

    ts_rank_volume = volume.rolling(window=32).apply(lambda x: (x.rank()[-1] - 1) / (len(x) - 1)).values[-1]
    ts_rank_price = ((close + high - low).rolling(window=16).apply(lambda x: (x.rank()[-1] - 1) / (len(x) - 1))).values[-1]
    ts_rank_returns = returns.rolling(window=32).apply(lambda x: (x.rank()[-1] - 1) / (len(x) - 1)).values[-1]

    alpha = ts_rank_volume * (1 - ts_rank_price) * (1 - ts_rank_returns)
    return alpha

In [341]:
def alpha_36(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    close = data['Close']
    open_ = data['Open']
    volume = data['Volume']
    returns = data['Close'].pct_change()
    adv20 = data['Volume'].rolling(window=20).mean()
    vwap = (data['Close'] * data['Volume']).cumsum() / data['Volume'].cumsum()

    rank_correlation = (2.21 * (close - open_).rolling(window=15).corr(volume.shift(-1))).rank(pct=True)
    rank_open_close = (0.7 * (open_ - close).rank(pct=True))
    rank_ts_rank_returns = (0.73 * returns.shift(-6).rolling(window=5).apply(lambda x: (x.rank()[-1] - 1) / (len(x) - 1))).rank(pct=True)
    rank_abs_correlation = abs(vwap.rolling(window=6).corr(adv20)).rank(pct=True)
    rank_sum_close = (0.6 * ((close.rolling(window=200).sum() / 200 - open_) * (close - open_))).rank(pct=True)

    alpha = rank_correlation + rank_open_close + rank_ts_rank_returns + rank_abs_correlation + rank_sum_close
    return alpha.dropna()

In [345]:
def alpha_37(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    open_ = data['Open']
    close = data['Close']

    # Calculate Alpha#37
    rank_correlation = (open_.shift(1) - close).rolling(window=200).corr(close).rank(pct=True)
    rank_open_close = (open_ - close).rank(pct=True)

    alpha = rank_correlation + rank_open_close
    return alpha.dropna()

In [359]:
def alpha_38(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    close = data['Close']
    open_ = data['Open']

    # Calculate Alpha#38
    ts_rank_close = close.rolling(window=10).apply(lambda x: np.argsort(x).argsort()[0], raw=True)
    rank_ts_rank_close = ts_rank_close.rank(pct=True)
    rank_close_open = (close / open_).rank(pct=True)

    alpha = -1 * rank_ts_rank_close * rank_close_open
    return alpha.dropna()

In [369]:
def alpha_39(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    close = data['Close']
    volume = data['Volume']

    # Calculate Alpha#39
    delta_close_7 = close.diff(7)
    adv20 = volume.rolling(window=20).mean()
    decay_linear_volume_adv20 = volume / adv20
    rank_decay_linear_volume_adv20 = decay_linear_volume_adv20.rolling(window=9).apply(lambda x: np.argsort(x).argsort()[0], raw=True).rank(pct=True)
    rank_delta_close_7 = delta_close_7.rank(pct=True)
    sum_returns_250 = (data['Close'].pct_change()).rolling(window=250).sum().rank(pct=True)

    alpha = (-1 * rank_delta_close_7 * (1 - rank_decay_linear_volume_adv20)) * (1 + sum_returns_250)
    return alpha.dropna()

In [411]:
def alpha_40(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate the standard deviation of high prices over a 10-day window
    stddev_high_10 = data['High'].rolling(window=10).std()

    # Calculate the correlation between high prices and volume over a 10-day window
    correlation_high_volume_10 = data['High'].rolling(window=10).corr(data['Volume'])

    # Calculate Alpha#40
    alpha_40 = (-1 * stddev_high_10.rank()) * correlation_high_volume_10

    return alpha_40.dropna()

In [381]:
def alpha_41(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate Alpha#41
    alpha_41 = (data['High'] * data['Low']) ** 0.5 - get_vwap(data)

    return alpha_41.dropna()

In [383]:
def alpha_42(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate VWAP
    vwap = get_vwap(data)

    # Calculate Alpha#42
    alpha_42 = ((vwap - data['Close']).rank() / (vwap + data['Close']).rank()).dropna()

    return alpha_42

In [385]:
def alpha_43(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate ADV20 (average daily volume over the last 20 days)
    data['ADV20'] = data['Volume'].rolling(window=20).mean()

    # Calculate Alpha#43
    alpha_43 = data['Volume'] / data['ADV20']
    ts_rank_1 = alpha_43.rolling(window=20).apply(lambda x: (pd.Series(x).rank(pct=True).iloc[-1] * 100))
    delta_close = -1 * data['Close'].diff(7)
    ts_rank_2 = delta_close.rolling(window=8).apply(lambda x: (pd.Series(x).rank(pct=True).iloc[-1] * 100))
    alpha_43 = ts_rank_1 * ts_rank_2

    return alpha_43.dropna()

In [389]:
def alpha_44(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate Alpha#44
    alpha_44 = -1 * data['High'].rolling(window=5).corr(data['Volume'].rank())

    return alpha_44.dropna()

In [412]:
def alpha_45(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate Alpha#45
    delay_close = data['Close'].shift(-5)
    sum_delay_close_20 = delay_close.rolling(window=20).sum()
    rank_1 = (sum_delay_close_20 / 20).rank()
    correlation_1 = data['Close'].rolling(window=2).corr(data['Volume'])
    rank_2 = correlation_1.rank()
    sum_close_5 = data['Close'].rolling(window=5).sum()
    sum_close_20 = data['Close'].rolling(window=20).sum()
    correlation_2 = sum_close_5.rolling(window=2).corr(sum_close_20)
    rank_3 = correlation_2.rank()
    alpha_45 = -1 * ((rank_1 * correlation_1) * rank_2 * rank_3)


    return alpha_45.dropna()

In [396]:
def alpha_46(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate Alpha#46
    condition = ((data['Close'].shift(20) - data['Close'].shift(-10)) / 10) - ((data['Close'].shift(10) - data['Close']) / 10)
    alpha_46 = np.where((0.25 < condition), -1, np.where((condition < 0), 1, -1 * (data['Close'] - data['Close'].shift(1))))

    return pd.Series(alpha_46, index=data.index).dropna()

In [400]:
def alpha_47(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate ADV20 (average daily volume over the last 20 days)
    data['ADV20'] = data['Volume'].rolling(window=20).mean()

    # Calculate Alpha#47
    rank_close = 1 / data['Close'].rank()
    alpha_47 = ((rank_close * data['Volume']) / data['ADV20']) * ((data['High'] * data['High'].rank()) / (data['High'].rolling(window=5).mean() / 5)) - (get_vwap(data) - get_vwap(data).shift(5)).rank()

    return alpha_47.dropna()

In [402]:
def alpha_48(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate Alpha#48
    delta_close_1 = data['Close'].diff(1)
    correlation_1 = delta_close_1.rolling(window=250).corr(data['Close'].shift(-1).diff(1))
    sum_1 = (delta_close_1 / data['Close'].shift(-1)) ** 2
    alpha_48 = indneutralize((correlation_1 * delta_close_1 / data['Close']) / sum_1.rolling(window=250).sum(), IndClass.subindustry) # I don't know what to do for indneutralize

    return alpha_48.dropna()

In [406]:
def alpha_49(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate Alpha#49
    condition = ((data['Close'].shift(-20) - data['Close'].shift(-10)) / 10) - ((data['Close'].shift(-10) - data['Close']) / 10) < (-1 * 0.1)
    alpha_49 = np.where(condition, 1, -1 * (data['Close'] - data['Close'].shift(-1)))

    return pd.Series(alpha_49, index=data.index).dropna()

In [409]:
def alpha_50(ticker, time_frame):
    # Download historical data
    data = yf.download(ticker, period=time_frame)

    # Calculate Alpha#50
    rank_volume = data['Volume'].rank()
    rank_vwap = get_vwap(data).rank()
    correlation_rank = rank_volume.rolling(window=5).corr(rank_vwap)
    alpha_50 = -1 * correlation_rank.rank(method='first').rolling(window=5).max()

    return alpha_50.dropna()

In [418]:
print("Printing alpha: ", alpha_26("GOOG", "5y"))
print("Printing alpha: ", alpha_27("GOOG", "5y"))
# print("Printing alpha: ", alpha_28("GOOG", "5y"))
print("Printing alpha: ", alpha_29("GOOG", "5y"))
print("Printing alpha: ", alpha_30("GOOG", "5y"))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Printing alpha:  -0.0
Printing alpha:  -1
Printing alpha:  628.347373493976
Date
2019-02-26           NaN
2019-02-27           NaN
2019-02-28   -915.000000
2019-03-01   -915.000000
2019-03-04   -915.000000
                 ...    
2024-02-16   -495.833333
2024-02-20   -496.166667
2024-02-21   -705.666667
2024-02-22   -915.000000
2024-02-23   -915.000000
Name: Close, Length: 1258, dtype: float64
Printing alpha:  Date
2019-03-22   -187.279897
2019-03-25   -134.655325
2019-03-26    -80.284351
2019-03-27    -71.498522
2019-03-28    -70.579169
                 ...    
2024-02-16   -106.550252
2024-02-20   -108.839358
2024-02-21   -151.530308
2024-02-22   -207.252275
2024-02-23   -177.499382
Length: 1240, dtype: float64


In [223]:
print("Printing alpha: ", alpha_29("AAPL", "1y"))

[*********************100%%**********************]  1 of 1 completed

Printing alpha:  125.54767932489452
